In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import string
import re
from sklearn import decomposition
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
#read data from kaggle
train = pd.read_csv('../input/train.tsv', sep='\t')
test = pd.read_csv('../input/test.tsv', sep='\t')

In [ ]:
#read data for local machine
curr_dir= os.getcwd()
train = pd.read_csv(curr_dir+"//Dataset/train.tsv", sep='\t')
test = pd.read_csv(curr_dir+"//Dataset/test.tsv",sep='\t')

In [13]:
#read csv file
curr_dir= os.getcwd()
train = pd.read_csv(curr_dir+"//Dataset/train_with_brand_from_name.csv",encoding = 'ISO-8859-1')
test = pd.read_csv(curr_dir+"//Dataset/test_with_brand_from_name.csv",encoding = 'ISO-8859-1')

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
train.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
0,1,Razer BlackWidow Chroma Keyboard,3,Razer,52.0,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts
1,2,AVA-VIV Blouse,1,Target,10.0,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse
2,6,Acacia pacific tides santorini top,3,Acacia Swimwear,64.0,0,Size small but straps slightly shortened to fi...,Women,Swimwear,Two-Piece
3,7,Girls cheer and tumbling bundle of 7,3,Soffe,6.0,1,You get three pairs of Sophie cheer shorts siz...,Sports & Outdoors,Apparel,Girls
4,8,Girls Nike Pro shorts,3,Nike,19.0,0,Girls Size small Plus green. Three shorts total.,Sports & Outdoors,Apparel,Girls


In [15]:
train.shape

(1482535, 10)

In [10]:
test.head()

,train_id,name,item_condition_id,brand_name,price,shipping,item_description,cat_1,cat_2,cat_3
727270,1269153,Nike Airforce 1s,3,Nike,21.0,0,Air force 1s size 5c,Kids,Boys 2T-5T,Shoes
1387652,1190191,Women's Leopard Flats,3,NaN,11.0,0,Good condition. Size 8.,Women,Shoes,Flats
969382,1277773,Silver ClipOn's Bamboo Earrings,1,Silver,6.0,1,These hallow Clip On Bamboo earrings are perfe...,Women,Jewelry,Earrings
1443771,1363228,Lularoe TC leggings,3,NaN,18.0,1,Normal wash wear Air dried,Women,Athletic Apparel,"Pants, Tights, Leggings"
834984,1456435,iPhone 6/6s Bottle case,1,Apple,9.0,1,iPhone 6/6s bottle case I have all colors in s...,Electronics,Cell Phones & Accessories,"Cases, Covers & Skins"


In [ ]:
'''def cat_split(row):
    try:
        text = row
        txt1, txt2, txt3 = text.split('/')
        return txt1, txt2, txt3
    except:
        return np.nan, np.nan, np.nan

train["cat_1"], train["cat_2"], train["cat_3"] = zip(*train.category_name.apply(lambda val: cat_split(val)))
test["cat_1"], test["cat_2"], test["cat_3"] = zip(*test.category_name.apply(lambda val: cat_split(val)))
'''

In [7]:
train.info()
print("------------------------------------------")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
brand_name           1000532 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
cat_1                1471819 non-null object
cat_2                1471819 non-null object
cat_3                1471819 non-null object
dtypes: float64(1), int64(3), object(6)
memory usage: 113.1+ MB
------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 9 columns):
test_id              693359 non-null int64
name                 693359 non-null object
item_condition_id    693359 non-null int64
brand_name           468187 non-null object
shipping             693359 non-null int64
item_

In [8]:
print(train.isnull().sum())
print ("---------------------------")
test.isnull().sum()

train_id                  0
name                      0
item_condition_id         0
brand_name           482003
price                     0
shipping                  0
item_description          4
cat_1                 10716
cat_2                 10716
cat_3                 10716
dtype: int64
---------------------------


test_id                   0
name                      0
item_condition_id         0
brand_name           225172
shipping                  0
item_description          0
cat_1                  5104
cat_2                  5104
cat_3                  5104
dtype: int64

In [ ]:
'''
train.drop(['category_name'], axis=1, inplace = True)
test.drop(['category_name'], axis=1, inplace = True)
'''

In [11]:
train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'test_id':'id'})

In [12]:
train['is_train'] = 1
test['is_train'] = 0

In [11]:
train_test_combine = pd.concat([train.drop(['price'],axis =1),test],axis = 0)

In [12]:
train_test_combine.head()

,id,name,item_condition_id,brand_name,shipping,item_description,cat_1,cat_2,cat_3,is_train
0,1,Razer BlackWidow Chroma Keyboard,3,Razer,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,1
1,2,AVA-VIV Blouse,1,Target,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,1
2,6,Acacia pacific tides santorini top,3,Acacia Swimwear,0,Size small but straps slightly shortened to fi...,Women,Swimwear,Two-Piece,1
3,7,Girls cheer and tumbling bundle of 7,3,Soffe,1,You get three pairs of Sophie cheer shorts siz...,Sports & Outdoors,Apparel,Girls,1
4,8,Girls Nike Pro shorts,3,Nike,0,Girls Size small Plus green. Three shorts total.,Sports & Outdoors,Apparel,Girls,1


In [13]:
train_test_combine['cat_3'].describe()

count                     2160074
unique                        876
top       Pants, Tights, Leggings
freq                        88077
Name: cat_3, dtype: object

In [ ]:
#train_test_combine = train_test_combine.drop(['cat_3'],axis=1)

In [14]:
train_test_combine = train_test_combine.sort_values("cat_1")

In [15]:
cat_1_list = list(train_test_combine['cat_1'].unique())

In [16]:
cat_1_list

['Beauty',
 'Electronics',
 'Handmade',
 'Home',
 'Kids',
 'Men',
 'Other',
 'Sports & Outdoors',
 'Vintage & Collectibles',
 'Women',
 nan]

In [17]:
df_1 = train_test_combine[train_test_combine['cat_1'] == 'Beauty']
df_2 = train_test_combine[train_test_combine['cat_1'] == 'Electronics']
df_3 = train_test_combine[train_test_combine['cat_1'] == 'Handmade']
df_4 = train_test_combine[train_test_combine['cat_1'] == 'Home']
df_5 = train_test_combine[train_test_combine['cat_1'] == 'Kids']
df_6 = train_test_combine[train_test_combine['cat_1'] == 'Other']
df_7 = train_test_combine[train_test_combine['cat_1'] == 'Sports & Outdoors']
df_8 = train_test_combine[train_test_combine['cat_1'] == 'Vintage & Collectibles']
df_9 = train_test_combine[train_test_combine['cat_1'] == 'Women']
df_10 = train_test_combine[train_test_combine['cat_1'].isnull()]

In [18]:
df_1.head()

,id,name,item_condition_id,brand_name,shipping,item_description,cat_1,cat_2,cat_3,is_train
330183,575768,5 Pair Red Cherry Lashes,1,Red Cherry,1,BNIB Those pictured are those included in this...,Beauty,Makeup,Eyes,1
471926,823019,Colourpop Lippie Stix,2,ColourPop Cosmetics,0,In the color BeautyCon Never used Shipping is ...,Beauty,Makeup,Lips,1
241829,421663,NEW! Bobbi Brown make up 7 set,1,Bobbi Brown,1,Eye opening Mascara Black .17/5ml Smokey Eye M...,Beauty,Makeup,Makeup Sets,1
285314,497484,NARS Dual,5,Nars,1,No description yet,Beauty,Makeup,Face,0
285315,497485,Morphe Palette,1,Morphe Cosmetics,1,*NEW AND AUTHENTIC* 35K,Beauty,Makeup,Makeup Palettes,0


In [19]:
def work_on_each_dataframe(df_w):
    #one hot encoding on one dataset
    df_w = pd.get_dummies(df_w, columns=["cat_1"])
    df_w = pd.get_dummies(df_w, columns=["cat_2"])
    df_w = pd.get_dummies(df_w, columns=["cat_3"])
    
    df_w.name = df_w.name.astype('category')
    #df_w.brand_name = df_w.brand_name.astype('category')
    
    #label Encoding
    #df_w.brand_name = df_w.brand_name.cat.codes
    df_w.name = df_w.name.cat.codes

    df_w = df_w.drop(['item_description'],axis = 1)
    
    return df_w

In [20]:
df_1 = work_on_each_dataframe(df_1)
df_2 = work_on_each_dataframe(df_2)
df_3 = work_on_each_dataframe(df_3)
df_4 = work_on_each_dataframe(df_4)
df_5 = work_on_each_dataframe(df_5)
df_6 = work_on_each_dataframe(df_6)
df_7 = work_on_each_dataframe(df_7)
df_8 = work_on_each_dataframe(df_8)
df_9 = work_on_each_dataframe(df_9)
df_10 = work_on_each_dataframe(df_10)

In [21]:
df_1.shape

(305093, 60)

In [22]:
def model_train(df_m):
    df_test= df_m.loc[df_m['is_train']==0]
    df_train= df_m.loc[df_m['is_train']==1]
    
    df_test = df_test.drop(['is_train'],axis=1)
    df_train = df_train.drop(['is_train'],axis=1)
    
    df_train['price'] = train.price
    #df_train['price'] = df_train['price'].apply(lambda x: np.log(x) if x>0 else x)
    return df_train,df_test

In [23]:
df_1_train, df_1_test = model_train(df_1)
df_2_train, df_2_test = model_train(df_2)
df_3_train, df_3_test = model_train(df_3)
df_4_train, df_4_test = model_train(df_4)
df_5_train, df_5_test = model_train(df_5)
df_6_train, df_6_test = model_train(df_6)
df_7_train, df_7_test = model_train(df_7)
df_8_train, df_8_test = model_train(df_8)
df_9_train, df_9_test = model_train(df_9)
df_10_train, df_10_test = model_train(df_10)

In [24]:
df_1_test.head()

,id,name,item_condition_id,brand_name,shipping,cat_1_Beauty,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoo & Conditioner Sets,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women
285314,497484,153262,5,Nars,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
285315,497485,151917,1,Morphe Cosmetics,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
285328,497508,234220,1,Younique,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
285312,497481,239111,2,Morphe Cosmetics,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
285345,497541,104990,1,IT Cosmetics,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
df_1_train.head()

,id,name,item_condition_id,brand_name,shipping,cat_1_Beauty,cat_2_Bath & Body,cat_2_Fragrance,cat_2_Hair Care,cat_2_Makeup,...,cat_3_Shampoo Plus Conditioner,cat_3_Shampoos,cat_3_Styling Products,cat_3_Styling Tools,cat_3_Sun,cat_3_Toiletry Kits,cat_3_Tweezers,cat_3_Waxing,cat_3_Women,price
330183,575768,15887,1,Red Cherry,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,12.0
471926,823019,65792,2,ColourPop Cosmetics,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,7.0
241829,421663,156614,1,Bobbi Brown,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,17.0
1318313,977450,51925,1,NaN,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,8.0
471930,823027,76395,1,Elizabeth Arden,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14.0


In [ ]:
'''
#PCA

#convert it to numpy arrays
X=df_train.values

#Scaling the values
X = scale(X)

pca = PCA(n_components=44)

pca.fit(X)

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)

'''

In [26]:
'''
brand_group: Dataframe comprising mean of each brand
brand_group1: Training Dataset, in which we will append the brand_weight
brand_group11: Testing Dataset, in which we will append the brand_weight
brand_group2: a temperory variable for train data
brand_group22: a temperory variable for test data 
'''
def brand_weight_train(df_train1,df_test1):
    brand_group = df_train1.groupby(['brand_name'])['price'].mean().to_frame()
    brand_group['brand_name'] =brand_group.index
    i=0
    j=0
    for index1,row1 in brand_group.iterrows():
        if i ==0:
            try:
                brand_group1 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group1.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                i=1
            except:
                pass
        else:
            try:
                brand_group2 = df_train1[df_train1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group2.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                brand_group1 = brand_group1.append(brand_group2)
            except:
                pass
        
        if j==0:
            try:
                brand_group11 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group11.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                j=1
            except:
                pass
            
        else:
            try:
                brand_group22 = df_test1[df_test1['brand_name'].str.contains(brand_group.loc[index1, 'brand_name'])==True]
                brand_group22.loc[:,'brand_weight'] = brand_group.loc[index1,'price']
                brand_group11 = brand_group11.append(brand_group22)
            except:
                pass
    
    return brand_group1, brand_group11


In [59]:
%%time
#df_1_train,df_1_test = brand_weight_train(df_1_train,df_1_test)
#df_2_train,df_2_test = brand_weight_train(df_2_train,df_2_test)
#df_3_train,df_3_test = brand_weight_train(df_3_train,df_3_test)
#df_4_train,df_4_test = brand_weight_train(df_4_train,df_4_test)
#df_5_train,df_5_test = brand_weight_train(df_5_train,df_5_test)
#df_6_train,df_6_test = brand_weight_train(df_6_train,df_6_test)
#df_7_train,df_7_test = brand_weight_train(df_7_train,df_7_test)
df_8_train,df_8_test = brand_weight_train(df_8_train,df_8_test)
df_9_train,df_9_test = brand_weight_train(df_9_train,df_9_test)
df_10_train,df_10_test = brand_weight_train(df_10_train,df_10_test)

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Wall time: 43min 43s


In [49]:
'''
df_2_train.drop(['brand_name'], axis=1, inplace = True)
df_2_test.drop(['brand_name'], axis=1, inplace = True)
'''

In [52]:
def encode_brand_name_price(df__train,df__test):
    df__train.brand_name = df__train.brand_name.astype('category')
    df__test.brand_name = df__test.brand_name.astype('category')
    df__train.brand_name = df__train.brand_name.cat.codes
    df__test.brand_name = df__test.brand_name.cat.codes
    df__train['price'] = df__train['price'].apply(lambda x: np.log(x) if x>0 else x)
    return df__train,df__test

In [60]:
df_1_train,df_1_test = encode_brand_name_price(df_1_train,df_1_test)
df_2_train,df_2_test = encode_brand_name_price(df_2_train,df_2_test)
#df_3_train,df_3_test = encode_brand_name_price(df_3_train,df_3_test)
#df_4_train,df_4_test = encode_brand_name_price(df_4_train,df_4_test)
#df_5_train,df_5_test = encode_brand_name_price(df_5_train,df_5_test)
#df_6_train,df_6_test = encode_brand_name_price(df_6_train,df_6_test)
#df_7_train,df_7_test = encode_brand_name_price(df_7_train,df_7_test)
df_8_train,df_8_test = encode_brand_name_price(df_8_train,df_8_test)
df_9_train,df_9_test = encode_brand_name_price(df_9_train,df_9_test)
df_10_train,df_10_test = encode_brand_name_price(df_10_train,df_10_test)

In [54]:
def Random_forest(df_train,df_test):
    x_train,y_train = df_train.drop(['price'],axis =1),df_train.price
    m = RandomForestRegressor(n_jobs=-1,min_samples_leaf=3,n_estimators=200)
    m.fit(x_train, y_train)
    
    print(m.score(x_train,y_train))

In [55]:
#Random_forest(df_1_train,df_1_test)
#Random_forest(df_2_train,df_2_test)
Random_forest(df_3_train,df_3_test)
Random_forest(df_4_train,df_4_test)
Random_forest(df_5_train,df_5_test)
#Random_forest(df_6_train,df_6_test)
#Random_forest(df_7_train,df_7_test)
#Random_forest(df_8_train,df_8_test)
#Random_forest(df_9_train,df_9_test)
#Random_forest(df_10_train,df_10_test)

0.857589853493
0.754778320467
0.771632284923


In [58]:
Random_forest(df_6_train,df_6_test)
Random_forest(df_7_train,df_7_test)

0.785997352692
0.777192130842


In [61]:
Random_forest(df_8_train,df_8_test)
Random_forest(df_9_train,df_9_test)
Random_forest(df_10_train,df_10_test)

0.797329625168
0.843613342433
0.860777646969


In [ ]:
preds = m.predict(df_test)

In [ ]:
preds = pd.Series(np.exp(preds))

In [ ]:
type(preds)

In [ ]:
submit = pd.concat([df_test.id,preds],axis=1)

In [ ]:
submit.columns = ['test_id','price']

In [ ]:
#df_1_train.to_csv(curr_dir+"//Dataset/df_1_train.csv")